In [5]:
import os
import re
import pandas as pd
import numpy as np

In [6]:
df = pd.read_parquet('../acts_parquet/aviso_licitacao_all_entities_iob.parquet')
df

,ORGAO_LICITANTE,NUM_LICITACAO,OBJ_LICITACAO,MODALIDADE_LICITACAO,PROCESSO,DATA_ABERTURA,TIPO_OBJ,VALOR_ESTIMADO,CODIGO_SISTEMA_COMPRAS,SISTEMA_COMPRAS,treated_text,IOB
0,None,01/2021,Contratação de Concessionária Autorizada pela ...,PREGÃO ELETRÔNICO,00052-00021170/2020-47,None,prestação de serviços,"R$ 826.897,20",926015,www.comprasnet.gov.br,AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO Nº 01/202...,O O O B-MODALIDADE_LICITACAO I-MODALIDADE_LICI...
1,None,50/2018,Contratação de empresa especializada para forn...,PREGÃO ELETRÔNICO,0052.0001119/2017,None,fornecimento de licenças de software,"8.568.972,28",926015,www.comprasnet.gov.br,AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO Nº 50/201...,O O O B-MODALIDADE_LICITACAO I-MODALIDADE_LICI...
2,None,14/2021,"Aquisição de lubrificantes (graxa, lubrificant...",PREGÃO ELETRÔNICO,092.041904/2020,01/02/2021,Aquisição,None,974200,www.comprasnet.gov.br,PREGÃO ELETRÔNICO Nº 14/2021 Processo: 092.041...,B-MODALIDADE_LICITACAO I-MODALIDADE_LICITACAO ...
3,None,02/2021,"A presente licitação tem como objeto, Registro...",PREGÃO ELETRÔNICO,00070-00004150/2020-93,21/01/2021,Registro de Preços,"R$ 1.346.049,80",None,WWwW.comprasgovernamentais.gov.br,AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO (SRP) Nº ...,O O O B-MODALIDADE_LICITACAO I-MODALIDADE_LICI...
4,None,144/2006,AQUISI- çÃo çÃo DE MATERIAL DE LIMPEZA E PRODU...,TOMADA DE PREÇOS,052.001.505/2005,10 de janeiro de 2006,AQUISI- çÃo çÃo,None,None,www.compras.df.gov.br,AVISO DE LICITAÇÃO TOMADA DE PREÇOS Nº 144/200...,O O O B-MODALIDADE_LICITACAO I-MODALIDADE_LICI...
...,...,...,...,...,...,...,...,...,...,...,...,...
229,None,442/2004,Aquisição de material de consumo farmacológico...,PREGÃO,None,24/08/2004,Aquisição,None,None,None,PREGÃO Nº 442/2004 SUCOM/SEF/DF Objeto: Aquisi...,B-MODALIDADE_LICITACAO O B-NUM_LICITACAO I-NUM...
230,None,451/2004,"Aquisição de medicamentos (Lamivudina, octreot...",PREGÃO,None,02/09/2004,Aquisição,None,None,None,PREGÃO Nº 451/2004 SUCOM/SEF/DF Objeto: Aquisi...,B-MODALIDADE_LICITACAO O B-NUM_LICITACAO I-NUM...
231,None,024/2002,aquisi- ção ção de conectores,Tomada de Preços de Material,None,10/02/2003,aquisi- ção ção,None,None,None,AVISO DE LICITAÇÃO A COMPANHIA ENERGÉTICA DE B...,O O O O O O O O O O O O O O O O O O O O O O O ...
232,None,005/2002,Aquisição de materiais permanentes para uso na...,TOMADA DE PREÇOS,053.000.918/2002,11/10/2002,Aquisição,None,None,None,AVISO DE LICITAÇÃO PROCESSO N.º 053.000.918/20...,O O O O O O B-PROCESSO I-PROCESSO O O B-MODALI...


In [7]:
df.isnull().sum()

ORGAO_LICITANTE           195
NUM_LICITACAO               2
OBJ_LICITACAO               1
MODALIDADE_LICITACAO        1
PROCESSO                   62
DATA_ABERTURA              29
TIPO_OBJ                   11
VALOR_ESTIMADO            166
CODIGO_SISTEMA_COMPRAS    191
SISTEMA_COMPRAS            67
treated_text                0
IOB                         0
dtype: int64

In [8]:
ORGAO_LICITANTE = []
VALOR_ESTIMADO = []
CODIGO_SISTEMA_COMPRAS = []


for i in range(len(df)):
    if df['ORGAO_LICITANTE'][i] != None:
        ORGAO_LICITANTE.append(i)
    if df['VALOR_ESTIMADO'][i] != None:
        VALOR_ESTIMADO.append(i)
    if df['CODIGO_SISTEMA_COMPRAS'][i] != None:
        CODIGO_SISTEMA_COMPRAS.append(i)

In [9]:
len(ORGAO_LICITANTE), len(VALOR_ESTIMADO), len(CODIGO_SISTEMA_COMPRAS)

(39, 68, 43)

In [10]:
ORGAO_LICITANTE_1 = ORGAO_LICITANTE[:(len(ORGAO_LICITANTE)*70)//100]
ORGAO_LICITANTE_2 = ORGAO_LICITANTE[(len(ORGAO_LICITANTE)*70)//100:]

VALOR_ESTIMADO_set = set(VALOR_ESTIMADO).difference(set(ORGAO_LICITANTE_2))
VALOR_ESTIMADO_1 = list(VALOR_ESTIMADO_set)[:(len(VALOR_ESTIMADO_set)*70)//100]
VALOR_ESTIMADO_2 = set(list(VALOR_ESTIMADO_set)[(len(VALOR_ESTIMADO_set)*70)//100:]).difference(set(ORGAO_LICITANTE_1))

CODIGO_SISTEMA_COMPRAS_set = set(CODIGO_SISTEMA_COMPRAS).difference(set(ORGAO_LICITANTE_2))
CODIGO_SISTEMA_COMPRAS_set = set(CODIGO_SISTEMA_COMPRAS_set).difference(set(VALOR_ESTIMADO_2))
CODIGO_SISTEMA_COMPRAS_1 = list(CODIGO_SISTEMA_COMPRAS_set)[:(len(CODIGO_SISTEMA_COMPRAS_set)*70)//100]
CODIGO_SISTEMA_COMPRAS_2 = set(list(CODIGO_SISTEMA_COMPRAS_set)[(len(CODIGO_SISTEMA_COMPRAS_set)*70)//100:]).difference(set(ORGAO_LICITANTE_1))
CODIGO_SISTEMA_COMPRAS_2 = set(list(CODIGO_SISTEMA_COMPRAS_set)[(len(CODIGO_SISTEMA_COMPRAS_set)*70)//100:]).difference(set(VALOR_ESTIMADO_1))


In [11]:
len(ORGAO_LICITANTE_1), len(ORGAO_LICITANTE_2), len(VALOR_ESTIMADO_1), len(VALOR_ESTIMADO_2), len(CODIGO_SISTEMA_COMPRAS_1), len(CODIGO_SISTEMA_COMPRAS_2)

(27, 12, 44, 15, 23, 3)

In [12]:
lic = []

lic.extend(ORGAO_LICITANTE_1)
lic.extend(VALOR_ESTIMADO_1)
lic.extend(CODIGO_SISTEMA_COMPRAS_1)

lic = sorted(list(set(lic)))

In [13]:
not_lic = []

not_lic.extend(ORGAO_LICITANTE_2)
not_lic.extend(VALOR_ESTIMADO_2)
not_lic.extend(CODIGO_SISTEMA_COMPRAS_2)

not_lic = sorted(list(set(not_lic)))

In [14]:
len(lic), len(not_lic)

(77, 30)

In [15]:
for i in lic:
    for j in not_lic:
        if i == j:
            print(i)

In [16]:
df_lic = df.iloc[lic]
df_not_lic = df.iloc[not_lic]

In [17]:
len(df_lic), len(df_not_lic)

(77, 30)

In [18]:
df_lic.isnull().sum()

ORGAO_LICITANTE           50
NUM_LICITACAO              2
OBJ_LICITACAO              0
MODALIDADE_LICITACAO       1
PROCESSO                   9
DATA_ABERTURA             15
TIPO_OBJ                   5
VALOR_ESTIMADO            29
CODIGO_SISTEMA_COMPRAS    46
SISTEMA_COMPRAS           18
treated_text               0
IOB                        0
dtype: int64

In [19]:
df_not_lic.isnull().sum()

ORGAO_LICITANTE           18
NUM_LICITACAO              0
OBJ_LICITACAO              1
MODALIDADE_LICITACAO       0
PROCESSO                   5
DATA_ABERTURA              4
TIPO_OBJ                   1
VALOR_ESTIMADO            10
CODIGO_SISTEMA_COMPRAS    18
SISTEMA_COMPRAS            9
treated_text               0
IOB                        0
dtype: int64

In [20]:
df_real = df.drop(df_lic.index)
df_real = df_real.drop(df_not_lic.index)

In [21]:
len(df_real)

127

In [22]:
train = pd.concat([df_lic, df_real[:int(len(df)*0.7) - len(df_lic)]])
test = pd.concat([df_real[int(len(df)*0.7) - len(df_lic):], df_not_lic])

In [23]:
len(train), len(test), len(df)

(163, 71, 234)

In [24]:
test.isnull().sum()

ORGAO_LICITANTE           59
NUM_LICITACAO              0
OBJ_LICITACAO              1
MODALIDADE_LICITACAO       0
PROCESSO                  19
DATA_ABERTURA              6
TIPO_OBJ                   2
VALOR_ESTIMADO            51
CODIGO_SISTEMA_COMPRAS    59
SISTEMA_COMPRAS           23
treated_text               0
IOB                        0
dtype: int64

In [25]:
train.isnull().sum()

ORGAO_LICITANTE           136
NUM_LICITACAO               2
OBJ_LICITACAO               0
MODALIDADE_LICITACAO        1
PROCESSO                   43
DATA_ABERTURA              23
TIPO_OBJ                    9
VALOR_ESTIMADO            115
CODIGO_SISTEMA_COMPRAS    132
SISTEMA_COMPRAS            44
treated_text                0
IOB                         0
dtype: int64

In [26]:
i = 0
for col in list(df.columns):
    print(f'{col}: {((len(train) - list(train.isnull().sum())[i]) * 100) / (len(df) - list(df.isnull().sum())[i])}%')
    i += 1

ORGAO_LICITANTE: 69.23076923076923%
NUM_LICITACAO: 69.39655172413794%
OBJ_LICITACAO: 69.95708154506438%
MODALIDADE_LICITACAO: 69.52789699570816%
PROCESSO: 69.76744186046511%
DATA_ABERTURA: 68.29268292682927%
TIPO_OBJ: 69.05829596412556%
VALOR_ESTIMADO: 70.58823529411765%
CODIGO_SISTEMA_COMPRAS: 72.09302325581395%
SISTEMA_COMPRAS: 71.25748502994011%
treated_text: 69.65811965811966%
IOB: 69.65811965811966%


In [27]:
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

In [28]:
train.to_parquet('train.parquet')
test.to_parquet('test.parquet')

In [29]:
train = pd.read_parquet('train.parquet')
test = pd.read_parquet('test.parquet')

len(train), len(test)

(163, 71)